In [1]:
"""
実行時に設定するパラメータ
"""

# Informerの学習パラメータ
# Informerを学習するときに使用したパラメータの文字列をここで代入する
ARG_STR = "--model informer --data ETTh1 --attn prob --freq h --features S --d_model 16 --d_ff 32"

# 学習済みのInformerモデルが保存されているパス
INFORMER_CKPT_PATH = "checkpoints/informer-small.pth"

In [2]:
import sys
sys.path.append("Informer2020")

import argparse
import random
import tqdm
import numpy as np
import torch
import pandas as pd

import dataset
from model.informer_model import InformerModel
from model.model import Model
from model.moment_model import MomentModel
from propose import ProposedModel
from evaluation import evaluate_mse, evaluate_nll

from main_informer import parse_args

In [3]:
def set_seed(seed: int) -> None:
    # random
    random.seed(seed)

    # numpy
    np.random.seed(seed)

    # pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.mps.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


def load_moment_model(args: argparse.Namespace) -> Model:
    return MomentModel(param="AutonLab/MOMENT-1-large", pred_len=args.pred_len)


def load_informer_model(args: argparse.Namespace) -> Model:
    return InformerModel(args, checkpoint_path=INFORMER_CKPT_PATH)


def load_proposed_model(moment_model: Model, informer_model: Model,input_size: int,train_dataset: torch.utils.data.Dataset, args: argparse.Namespace) -> Model:
    model = ProposedModel(moment_model=moment_model, informer_model=informer_model,input_size=input_size)
    model.train(train_dataset=train_dataset,args=args)
    return model

In [ ]:
set_seed(0)

args = parse_args(ARG_STR)
print("args:", args)

train_dataset, test_dataset = dataset.load_dataset(args=args)
input_size = args.seq_len
moment_model = load_moment_model(args=args)
informer_model = load_informer_model(args=args)

In [5]:
proposed_model = load_proposed_model(moment_model, informer_model,input_size,test_dataset,args)

In [ ]:
# 予測結果の評価
results = {}

for method, model in {
    "informer": informer_model,
    "moment": moment_model,
    "proposed": proposed_model,
}.items():
    print(f"testing: {method}")
    test_dataloader = dataset.to_dataloader(test_dataset, args, "test")

    y_pred = []
    y_true = []
    for batch in tqdm.tqdm(test_dataloader):
        y_pred.append(model.predict_distr(batch).detach().tolist())
        y_true.append(batch[1][:, -1].squeeze().detach().tolist())

    y_pred, y_true = np.array(y_pred).reshape(-1, 2), np.array(y_true).flatten()
    results[method] = {
        "mse": evaluate_mse(y_pred, y_true),
        "nll": evaluate_nll(y_pred, y_true),
    }
    print(results[method])

    np.save(f"checkpoints/y_pred_{method}.npy", y_pred)

results

In [ ]:
pd.DataFrame(results).to_csv("results.csv", index=False)